In [1]:
from pprint import pprint

from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.datapipeline.etl.ingestion.reader.byblock import construct_readers_byblock
from op_analytics.datapipeline.etl.ingestion.reader.request import BlockBatchRequest
from op_analytics.datapipeline.models.compute.modelspec import ModelsDataSpec
from op_analytics.datapipeline.models.compute.testutils import setup_execution_context

In [ ]:
model_name = "token_transfers"


# Prepare data raeders
data_spec = ModelsDataSpec(models=[model_name],
    root_path_prefix="blockbatch")
blockbatch_request = BlockBatchRequest.build(
    chains=["op"],
    range_spec="@20241118:+1",
    root_paths_to_read=data_spec.input_root_paths,
)
readers: list[DataReader] = construct_readers_byblock(
    blockbatch_request=blockbatch_request,
    read_from=DataLocation.GCS,
)


# Show details for the batch we are processing.
pprint(readers[0])

# Set up execution context and get handles to model input args.
# In subsequent cells you can use the model input args however you want.
ctx, input_datasets, auxiliary_templates = setup_execution_context(
    model_name=model_name,
    data_reader=readers[0],  # use the first reader
)

In [ ]:
print(ctx.client.sql("SHOW TABLES").df())

In [ ]:
logs_view = input_datasets["ingestion/logs_v1"].create_view()

all_transfers = auxiliary_templates["token_transfers"].to_relation(
    duckdb_context=ctx,
    template_parameters={
        "raw_logs": logs_view,
    },
)

In [ ]:
df = ctx.client.sql(f"SELECT * FROM {logs_view} as l where l.topic0 LIKE '0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef%' LIMIT 10").df()
df.head()

In [ ]:
erc20_transfers = (
    all_transfers
    .filter("token_id IS NULL")
    .project("* EXCLUDE token_id")
)

df = ctx.client.sql(f"SELECT * FROM erc20_transfers").df()
df.head()

In [ ]:
erc721_transfers = (
    all_transfers
    .filter("token_id IS NOT NULL")
    .project("* EXCLUDE (amount, amount_lossless)")
)
df = ctx.client.sql(f"SELECT * FROM erc721_transfers").df()
df.head()